# Style encoding from Xander Steenbrugge
https://www.youtube.com/watch?v=dCKbRCUyop8

## Part I: Encoding images into StyleGAN's latent space

Make sure the Runtime type has GPU acceleration enabled

In [ ]:
import psutil
import os
import numpy as np
import humanize
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
printm() 

In [ ]:
import tensorflow as tf
tf.VERSION
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
!nvcc --version
!nvidia-smi

In [ ]:
!pwd

In [ ]:
!ls

## set globals

In [ ]:
uploadRawImages = 1
takePhotoImages = 0

### Clone pbaylies' stylegan encoder:

In [ ]:
path = 'stylegan-encoder'
isExist = os.path.exists(path)
if not isExist:
  print("stylegan-encoder does not exist. to clone")
  !git clone https://github.com/pbaylies/stylegan-encoder
else:
  print("stylegan-encoder already exists. will not clone")

### cd into the repo folder: (only run this cell once or things might get buggy)

In [ ]:
cd stylegan-encoder

### Let's see the files inside the repo we just cloned:

In [ ]:
ls


### create aligned_images folder

In [ ]:
mkdir -p aligned_images

##create raw_images folder

In [ ]:
mkdir -p raw_images

# I. Get Images:

## Some tips for the images:


*   Use HD images (preferably > 1000x1000 pixels)
*   Make sure your face is not too small
*   Neutral expressions & front facing faces will give better results
*   Clear, uniform lighting conditions are also recommened



## Option 1: Upload Images manually (usually gives the best results)



*   Click the '>' icon in the panel on the top left 
*   Go to the 'Files' tab
*   Unfold the stylegan-encoder folder
*   Right click the 'stylegan-encoder/raw_images' folder and click "upload"
*   I'd recommend starting with 3 - 6 different images



In [ ]:
!pwd

In [ ]:
if uploadRawImages == 1:
  !cd raw_images # will not change cwd
  print("upload raw images to stylegan-encoder/raw_image")
  from google.colab import files
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

In [ ]:
!pwd

## Option 2: Take images using your webcam

In [ ]:
if takePhotoImages == 1:
  from base64 import b64decode
  from IPython.display import HTML, Audio
  from google.colab.output import eval_js
  from PIL import Image
  from datetime import datetime

  VIDEO_HTML = """
  <video autoplay
   width=%d height=%d style='cursor: pointer;'></video>
  <script>

  var video = document.querySelector('video')

  navigator.mediaDevices.getUserMedia({ video: true })
    .then(stream=> video.srcObject = stream)

  var data = new Promise(resolve=>{
    video.onclick = ()=>{
      var canvas = document.createElement('canvas')
      var [w,h] = [video.offsetWidth, video.offsetHeight]
      canvas.width = w
      canvas.height = h
      canvas.getContext('2d')
            .drawImage(video, 0, 0, w, h)
      video.srcObject.getVideoTracks()[0].stop()
      video.replaceWith(canvas)
      resolve(canvas.toDataURL('image/jpeg', %f))
    }
  })
  </script>
  """

  def take_photo(quality=1.0, size=(800,600)):
    display(HTML(VIDEO_HTML % (size[0],size[1],quality)))
    data = eval_js("data")
    binary = b64decode(data.split(',')[1])
    f = io.BytesIO(binary)
    img = np.asarray(Image.open(f))

    timestampStr = datetime.now().strftime("%d-%b-%Y (%H:%M:%S.%f)")
    filename = 'raw_images/photo_%s.jpeg' %timestampStr
    Image.fromarray(img).save(filename)
    print('Image captured and saved to %s' %filename)

  img = take_photo() # click the image to capture a frame!

## Let's check the contents of our image folder before we start:
#### (You can always manually delete images by right clicking on them in the file tab)

In [ ]:
from PIL import Image
import os
imgs = sorted(os.listdir('raw_images'))

print("Found %d images in %s" %(len(imgs), 'raw_images'))
if len(imgs) == 0:
  print("Upload images to the \"raw_images\" folder!")
else:
  print(imgs)

for img_path in imgs:
  img = Image.open('raw_images/' + img_path)
  
  w,h = img.size
  rescale_ratio = 256 / min(w,h)
  img = img.resize((int(rescale_ratio*w),int(rescale_ratio*h)), Image.LANCZOS)
  display(img)

# II. Auto-Align faces:
### This script wil:


1.   Look for faces in the images
2.   Crop out the faces from the images
3.   Align the faces (center the nose and make the eyes horizontal)
4.   Rescale the resulting images and save them in "aligned_images" folder

### The cell below takes about a minute to run



In [ ]:
!python align_images.py raw_images/ aligned_images/ --output_size=1048

## Let's take a look at our aligned images:

In [ ]:
def display_folder_content(folder, res = 256):
  if folder[-1] != '/': folder += '/'
  for i, img_path in enumerate(sorted(os.listdir(folder))):
    if '.png' in img_path:
      display(Image.open(folder+img_path).resize((res,res)), 'img %d: %s' %(i, img_path))
      print('\n')
      
display_folder_content('aligned_images')

# Important, before moving on:
### Manually clean the 'aligned_images' directory

> ### 1. Manually remove all 'bad' images that are not faces / don't look sharp / clear 
> #####  (Use the image names from the plots above to guide you)
> ### 2. Make sure you don't have too many faces in this folder (8 at most preferably)




### Encoding faces into StyleGAN latent space:

### We'll be using pbaylies' awesome encoder repo (building on original work from Puzer): https://github.com/pbaylies/stylegan-encoder


## First, let's download a pretrained resnet encoder: (see video for what this does)
### --> This model takes an image as input and estimates the corresponding latent code

In [ ]:
!gdown https://drive.google.com/uc?id=1aT59NFy9-bNyXjDuZOTMl0qX0jmZc6Zb
!mkdir -p data
!mv finetuned_resnet.h5 data
!rm -rf generated_images latent_representations

# III. The actual encoding process:
> #### Play with the encoding params: they have a huge effect on the latent representations & images!
> #### Extra encoding options: https://github.com/pbaylies/stylegan-encoder/blob/master/encode_images.py

#### Note: This script will also download:


*   The pretrained StyleGAN network from NVIDIA trained on faces
*   A pretrained VGG-16 network, trained on ImageNet

#### After guessing the initial latent codes using the pretrained ResNet, it will run gradient descent to optimize the latent faces!
#### Note that by default, we're optimizing w vectors, not z-vectors!


In [ ]:
print("aligned_images contains %d images ready for encoding!" %len(os.listdir('aligned_images/')))

In [ ]:
rm -r videos # remove videos

#### For best performance, set the batch_size argument below equal to the number of aligned_images (see previous cell)
> Keep batch_size<8 or the GPU might run out of memory (Colab runs on Tesla K80's)

### Depending on the settings, the encoding process might take a few minutes...

In [ ]:
!python encode_images.py \
--batch_size=2 \
--output_video=True \
--load_resnet='data/finetuned_resnet.h5' \
--lr=0.01 \
--decay_rate=0.8 \
--iterations=200 \
--use_l1_penalty=0.3 aligned_images/ generated_images/ latent_representations/

print("\n************ Latent code optimization finished! ***************")

## Showtime!
### Let's load the StyleGAN network into memory:

In [ ]:
import dnnlib, pickle
import dnnlib.tflib as tflib
tflib.init_tf()
synthesis_kwargs = dict(output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True), minibatch_size=1)

model_dir = 'cache/'
model_path = [model_dir+f for f in os.listdir(model_dir) if 'stylegan-ffhq' in f][0]
print("Loading StyleGAN model from %s..." %model_path)

with dnnlib.util.open_url(model_path) as f:
  generator_network, discriminator_network, averaged_generator_network = pickle.load(f)
  
print("StyleGAN loaded & ready for sampling!")

In [ ]:
def generate_images(generator, latent_vector, z = True):
    batch_size = latent_vector.shape[0]
    
    if z: #Start from z: run the full generator network
        return generator.run(latent_vector.reshape((batch_size, 512)), None, randomize_noise=False, **synthesis_kwargs)
    else: #Start from w: skip the mapping network
        return generator.components.synthesis.run(latent_vector.reshape((batch_size, 18, 512)), randomize_noise=False, **synthesis_kwargs)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def plot_imgs(model, rows, columns):
  for i in range(rows):
    f, axarr = plt.subplots(1,columns, figsize = (20,8))
    for j in range(columns):
      img = generate_images(model, np.random.randn(1,512), z = True)[0]
      axarr[j].imshow(img)
      axarr[j].axis('off')
      axarr[j].set_title('Resolution: %s' %str(img.shape))
    plt.show()

##Let's plot some random StyleGAN samples:

In [ ]:
plot_imgs(averaged_generator_network, 3, 3)

# Let's take a look at the results of our encoding:
### If the results don't look great: Play with the encoding arguments!!!
> 1. Run the optimization for more iterations (eg 1000)
> 2. Decrease the L1 penalty (to eg 0.1)
> 3. Try a lower initial learning rate (eg 0.005) or play with the decay_rate
> 4. Find out about the other encoding options here: https://github.com/pbaylies/stylegan-encoder/blob/master/encode_images.py

In [ ]:
import numpy as np

for f in sorted(os.listdir('latent_representations')):
  w = np.load('latent_representations/' + f).reshape((1,18,-1))
  img = generate_images(averaged_generator_network, w, z = False)[0]
  plt.imshow(img)
  plt.axis('off')
  plt.title("Generated image from %s" %f)
  plt.show()

## Let's compare our encoded samples with the original ones:

In [ ]:
import matplotlib.pyplot as plt

def plot_two_images(img1,img2, img_id, fs = 12):
  f, axarr = plt.subplots(1,2, figsize=(fs,fs))
  axarr[0].imshow(img1)
  axarr[0].title.set_text('Encoded img %d' %img_id)
  axarr[1].imshow(img2)
  axarr[1].title.set_text('Original img %d' %img_id)
  plt.setp(plt.gcf().get_axes(), xticks=[], yticks=[])
  plt.show()

def display_sbs(folder1, folder2, res = 256):
  if folder1[-1] != '/': folder1 += '/'
  if folder2[-1] != '/': folder2 += '/'
    
  imgs1 = sorted([f for f in os.listdir(folder1) if '.png' in f])
  imgs2 = sorted([f for f in os.listdir(folder2) if '.png' in f])
  if len(imgs1)!=len(imgs2):
    print("Found different amount of images in aligned vs raw image directories. That's not supposed to happen...")
  
  for i in range(len(imgs1)):
    img1 = Image.open(folder1+imgs1[i]).resize((res,res))
    img2 = Image.open(folder2+imgs2[i]).resize((res,res))
    plot_two_images(img1,img2, i)
    print("")
     
display_sbs('generated_images/', 'aligned_images/', res = 512)

### Note: 
If you want to watch the whole thing unfold for yourself, you can **download the optimization videos** from the "videos" folder

In [ ]:
ls videos

In [ ]:
videos_dir = 'videos'  
videos_dir_bck = 'videos'  


!ls $videos_dir

thereisbck = os.path.isdir(videos_dir_bck)
print("thereisbck %s" %(thereisbck))

if thereisbck == True:
  print("video folders with first names: %s" %(videos_dir_bck))
else:
  print("copy to bck")
  !cp -r $videos_dir $videos_dir_bck

vids = sorted(os.listdir(videos_dir))
print("Found %d videos in %s" %(len(vids), videos_dir))
for i, vid_fullname in enumerate(vids):
  vid_name = vid_fullname.split('.')[0]
  vid_ext = vid_fullname.split('.')[1]
  newvid_fullname = 'vid_%02d.%s' %(i, vid_ext)
  os.rename(videos_dir + '/' + vid_fullname, videos_dir + '/' + newvid_fullname)
!ls $videos_dir

In [ ]:
from moviepy.editor import *
from moviepy.video.io.VideoFileClip import VideoFileClip
from moviepy.Clip import Clip
from IPython.display import display

videos_dir = 'videos'  
vids = sorted(os.listdir(videos_dir))
for i, vid in enumerate(vids):
  vid_name = videos_dir + '/' + vid
  print("vid %d : %s" %(i, vid_name))
  clip = VideoFileClip(vid_name)
  display(clip.ipython_display(height=512, autoplay=1, loop=1))


# IV. Set the good images & dump their latent vectors to disk



In [ ]:
good_images = []
folder = 'latent_representations'
for i, latents in enumerate(sorted(os.listdir(folder))):
  good_images.append(i)
      
# good_images = [0,1,2,3]  # Uncomment and Change these numbers to pick out latents that worked well (see the image plots)

In [ ]:
import numpy as np
latents = sorted(os.listdir('latent_representations'))

out_file = '../output_vectors.npy'

final_w_vectors = []
for img_id in good_images:
  w = np.load('latent_representations/' + latents[img_id])
  final_w_vectors.append(w)

final_w_vectors = np.array(final_w_vectors)
np.save(out_file, final_w_vectors)
print("%d latent vectors of shape %s saved to %s!" %(len(good_images), str(w.shape), out_file))

In [ ]:
from google.colab import files
files.download('../output_vectors.npy')

# Notebook II: Playing with Latent Codes

In [ ]:
import os

def restart_runtime():
  os.kill(os.getpid(), 9)

In [ ]:
!pwd
!ls

In [ ]:
import tensorflow as tf
tf.VERSION

In [ ]:
!nvcc --version

## Clone tr1pzz's fork of original ShenYujun's InterFaceGAN
### Original Repo: https://github.com/ShenYujun/InterFaceGAN
### Paper: https://arxiv.org/abs/1907.10786

In [ ]:
cd ../

In [ ]:
path = 'InterFaceGAN'
isExist = os.path.exists(path)
if not isExist:
  print("InterFaceGAN does not exist. to clone")
  !git clone https://github.com/tr1pzz/InterFaceGAN.git
else:
  print("InterFaceGAN already exists. will not clone")

In [ ]:
cd InterFaceGAN/

## Download the pretrained StyleGAN FFHQ network from NVIDIA:

In [ ]:
!gdown https://drive.google.com/uc?id=1MEGjdvVpUsu1jB4zrXZN7Y4kBBOzizDQ

In [ ]:
!mv karras2019stylegan-ffhq-1024x1024.pkl models/pretrain/karras2019stylegan-ffhq-1024x1024.pkl

# I. Let's load the latent space vectors:


In [ ]:
!ls ../output_vectors.npy

In [ ]:
import numpy as np
final_w_vectors = np.load('../output_vectors.npy')

print("%d latent vectors of shape %s loaded from %s!" %(final_w_vectors.shape[0], str(final_w_vectors.shape[1:]), 'output_vectors.npy'))

## The InterFaceGAN comes with a bunch of pretrained latent directions
### (However, you can also train your own!!)
### Pick the latent space manipulation we want to use (added it as the -b argument below)

Boundaries: https://github.com/ShenYujun/InterFaceGAN/tree/master/boundaries
* stylegan_ffhq_age_w_boundary.npy
* stylegan_ffhq_eyeglasses_w_boundary.npy
* stylegan_ffhq_gender_w_boundary.npy
* stylegan_ffhq_pose_w_boundary.npy
* stylegan_ffhq_smile_w_boundary.npy


# II. Let's configure our latent-space interpolation
### Change the settings below to morph the faces:

In [ ]:
latent_direction = 'age'     #### Pick one of ['age', 'eyeglasses', 'gender', 'pose', 'smile']
morph_strength = 3           # Controls how strongly we push the face into a certain latent direction (try 1-5)
nr_interpolation_steps = 48  # The amount of intermediate steps/frames to render along the interpolation path

# III. Run the latent space manipulation & generate images:

In [ ]:
boundary_file = 'stylegan_ffhq_%s_w_boundary.npy' %latent_direction

print("Ready to start manipulating faces in the ** %s ** direction!" %latent_direction)
print("Interpolation from %d to %d with %d intermediate frames." %(-morph_strength, morph_strength, nr_interpolation_steps))
print("\nLoading latent directions from %s" %boundary_file)

## Ready? Set, Go!

In [ ]:
import subprocess
return_code = subprocess.call("rm -r results/%s" %latent_direction, shell=True)

run_command = "python edit.py \
      -m stylegan_ffhq \
      -b boundaries/stylegan_ffhq_%s_w_boundary.npy \
      -s Wp \
      -i '../output_vectors.npy' \
      -o results/%s \
      --start_distance %.2f \
      --end_distance %.2f \
      --steps=%d" %(latent_direction, latent_direction, -morph_strength, morph_strength, nr_interpolation_steps)


print("Running latent interpolations... This should not take longer than ~1 minute")
print("Running: %s" %run_command)
return_code = subprocess.call(run_command, shell=True)

if not return_code:
  print("Latent interpolation successfully dumped to disk!")
else:
  print("Something went wrong, try re-executing this cell...")

In [ ]:
# cd /content/InterFaceGAN/

In [ ]:

!rm -rf results/age
!python edit.py \
  -m stylegan_ffhq \
  -b boundaries/stylegan_ffhq_age_w_boundary.npy \
  -s Wp \
  -i '../output_vectors.npy' \
  -o results/age \
  --start_distance -3.0 \
  --end_distance 3.0 \
  --steps=48

# IV. Finally, turn the results into pretty movies!
Adjust which video to render & at what framerate:

In [ ]:
image_folder = 'results/%s' %latent_direction
video_fps = 12.

### Render the videos:

In [ ]:
from moviepy.editor import *
import cv2

out_path = 'output_videos/'

images = [img_path for img_path in sorted(os.listdir(image_folder)) if '.jpg' in img_path]
os.makedirs(out_path, exist_ok=True)

prev_id = None
img_sets = []
for img_path in images:
  img_id = img_path.split('_')[0]
  if img_id == prev_id: #append
    img_sets[-1].append(img_path)
    
  else: #start a new img set
    img_sets.append([])
    img_sets[-1].append(img_path)
  prev_id = img_id

print("Found %d image sets!\n" %len(img_sets))
if image_folder[-1] != '/':
  image_folder += '/'

def make_video(images, vid_name):
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter(vid_name, fourcc, video_fps, (1024, 1024))
    gen = {}
    for img in images:
      video.write(img)
    video.release()
    print('finished '+ vid_name)
    
    
for i in range(len(img_sets)):
  print("\nGenerating video %d..." %i)
  set_images = []
  vid_name = out_path + 'out_video_%s_%02d.mp4' %(latent_direction,i)
  for img_path in img_sets[i]:
    set_images.append(cv2.imread(image_folder + img_path))

  set_images.extend(reversed(set_images))
  make_video(set_images, vid_name)

## Visualise the resulting videos inside this Notebook:

In [ ]:
video_file_to_show = 0

clip = VideoFileClip('output_videos/out_video_%s_%02d.mp4' %(latent_direction, video_file_to_show))
clip.ipython_display(height=512, autoplay=1, loop=1)

In [ ]:
video_file_to_show = 1

clip = VideoFileClip('output_videos/out_video_%s_%02d.mp4' %(latent_direction, video_file_to_show))
clip.ipython_display(height=512, autoplay=1, loop=1)

# V. Your turn to experiment

## You now have all the tools to start exploring the latent Space of StyleGAN: HAVE FUN!
### StyleGAN paper link: https://arxiv.org/abs/1812.04948

### Some things you could try:
* You can blend between two faces by doing a linear interpolation in the latent space: very cool!
*   The StyleGAN vector has 18x512 dimensions, each of those 18 going into a different layer of the generator...
*   You could eg take the first 9 from person A and the next 9 from person B
*   This is why it's called "Style-GAN": you can manipulate the style of an image at multiple levels of the Generator!
*   Try interpolating in Z-space rather than in W-space (see InterFaceGan paper & repo)
* Have Fun!!
# Find something cool you wanna share? 
## ---> Tag me on Twitter @xsteenbrugge: https://twitter.com/xsteenbrugge
## ---> Or simply share it in the comments on YouTube!